<a href="https://colab.research.google.com/github/sheikmohdimran/Experiments_2019/blob/master/CADEC_NLP_SCIBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#In Python2
#!unzip -q CADEC.zip
#!rm CADEC.zip; rm CADEC.v1.zip; rm -rf CSIRO_DAP_Metadata
#!unzip CADEC.v2.zip
#!mv cadec/original/* cadec/text/
#!cp  cadec/text/DICLOFENAC-SODIUM.7.ann  cadec/text/DICLOFENAC-SODIUM.7.ann.orig
#!sed -i 's/421/411/g'  cadec/text/DICLOFENAC-SODIUM.7.ann  #Change annotation lines
#!rm -rf VOLTAREN-XR.9*; rm -rf LIPITOR.40* #No entries with these files
#!git clone https://github.com/spyysalo/standoff2conll
#!python standoff2conll/standoff2conll.py -i -s IO cadec/text  > cadec/text.tsv

In [0]:
import pandas as pd
import numpy as np

with open('cadec_text.tsv') as f:
    content = f.readlines()

indices = [i for i, x in enumerate(content) if x.startswith("# doc_id")]
df1 = pd.DataFrame([sub.split("\t") for sub in content], columns=('words','tags'))
indices1 = [i for i, x in enumerate(content) if x.startswith("\n")]
a=0
indices1.insert(0, a)
k=0
j = None
for i in indices1:
    if j != None:
      df1.loc[j:i, ("doc_id")] = k
    j = i
    k += 1

df1 = df1.replace('\n','', regex=True)
df1 = df1.replace('I-Symptom','O')
df1 = df1.replace('I-Disease','O')
df1 = df1.replace('I-Finding','O')
df1 = df1.drop(df1.index[list(indices)])
df1.dropna(inplace=True)
df1 = df1.reset_index(drop=True)

In [0]:
from tqdm import tqdm, trange

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["words"].values.tolist(),
                                                           s["tags"].values.tolist())]
        self.grouped = self.data.groupby("doc_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None


In [0]:
getter = SentenceGetter(df1)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
labels = [[s[1] for s in sent] for sent in getter.sentences]
tags_vals = list(set(df1["tags"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}



In [0]:
!pip install pytorch-pretrained-bert==0.4.0

     |████████████████████████████████| 51kB 2.5MB/s 


In [0]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
MAX_LEN = 75
bs = 32

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 2674154.54B/s]


In [0]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)


In [0]:
#!wget https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar
!tar -xvf scibert_scivocab_uncased.tar

scibert_scivocab_uncased/
scibert_scivocab_uncased/weights.tar.gz
scibert_scivocab_uncased/vocab.txt


In [0]:
!find ../ -name "bert-base-uncased.tar.gz"

In [0]:
PRETRAINED_MODEL_ARCHIVE_MAP = {'bert-base-uncased': "/content/scibert_scivocab_uncased/weights.tar.gz"}
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))
model.cuda();


In [0]:
#!wget https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar
#!tar -xvf scibert_scivocab_uncased.tar

In [0]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)


In [0]:
!pip install seqeval

In [0]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
epochs = 5
max_grad_norm = 1.0
for _ in trange(epochs, desc="Epoch"):
  # TRAIN loop
  model.train()
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  for step, batch in enumerate(train_dataloader):
    # add batch to gpu
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    # forward pass
    loss = model(b_input_ids, token_type_ids=None,
    attention_mask=b_input_mask, labels=b_labels)
    # backward pass
    loss.backward()
    # track train loss
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
    # gradient clipping
    torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
    # update parameters
    optimizer.step()
    model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch
      with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
        eval_loss = eval_loss/nb_eval_steps
        print("Validation loss: {}".format(eval_loss))
        print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
        pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
        valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
        print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.9152888059616089
Validation loss: 0.5186611413955688
Validation Accuracy: 0.9641666666666666
F1-Score: 0
Validation loss: 0.5072627812623978
Validation Accuracy: 0.97
F1-Score: 0
Validation loss: 0.3395175983508428
Validation Accuracy: 0.9706944444444444
F1-Score: 0
Validation loss: 0.21947363143165904
Validation Accuracy: 0.966875
F1-Score: 0
Validation loss: 0.15043844605485596
Validation Accuracy: 0.9672499999999999
F1-Score: 0
Validation loss: 0.11285252674586244
Validation Accuracy: 0.9677777777777777
F1-Score: 0
Validation loss: 0.0917898520176846
Validation Accuracy: 0.9678571428571429
F1-Score: 0
Validation loss: 0.0783342955552692
Validation Accuracy: 0.9680729166666667
F1-Score: 0
Validation loss: 0.06709424255118496
Validation Accuracy: 0.9683796296296298
F1-Score: 0
Validation loss: 0.05988328218244088
Validation Accuracy: 0.9687083333333334
F1-Score: 0
Validation loss: 0.05317077723396779
Validation Accuracy: 0.9696969696969698
F1-Score: 0
Validation loss: 0.